# Leakage characterization using GST
This tutorial demonstrates how to perform GST on a "leaky-qubit" described by a 3-level (instead of the desired 2-level) system. 

In [1]:
import pygsti
import pygsti.construction as pc
import pygsti.modelpacks.legacy.std1Q_XYI as std1Q
import numpy as np
import scipy.linalg as sla
#import pickle

In [2]:
def to_3level_unitary(U_2level):
    U_3level = np.zeros((3,3),complex)
    U_3level[0:2,0:2] = U_2level
    U_3level[2,2] = 1.0
    return U_3level

def unitary_to_gmgate(U):
    return pygsti.tools.change_basis( 
        pygsti.tools.unitary_to_process_mx(U), 'std','gm')

def state_to_gmvec(state):
    pygsti.tools.stdmx_to_gmvec

Us = pygsti.tools.internalgates.standard_gatename_unitaries()

In [3]:
mdl_2level_ideal = std1Q.target_model()

In [4]:
rho0 = np.array( [[1,0,0],
                  [0,0,0],
                  [0,0,0]], complex)
E0 = rho0
E1 = np.array( [[0,0,0],
                [0,1,0],
                [0,0,1]], complex)

sslbls = pygsti.obj.StateSpaceLabels(['Qubit+Leakage'],[9])
mdl_3level_ideal = pygsti.obj.ExplicitOpModel(sslbls, 'gm')
mdl_3level_ideal['rho0'] = pygsti.tools.stdmx_to_gmvec(rho0)
mdl_3level_ideal['Mdefault'] = pygsti.obj.TPPOVM([('0',pygsti.tools.stdmx_to_gmvec(E0)),
                                                    ('1',pygsti.tools.stdmx_to_gmvec(E1))])

mdl_3level_ideal['Gi'] = unitary_to_gmgate( to_3level_unitary(Us['Gi']))
mdl_3level_ideal['Gx'] = unitary_to_gmgate( to_3level_unitary(Us['Gxpi2']))
mdl_3level_ideal['Gy'] = unitary_to_gmgate( to_3level_unitary(Us['Gypi2']))

In [5]:
sigmaX = np.array([[0,1],[1,0]],complex)
rot = sla.expm(1j * 0.1 * sigmaX)
Uleakage = np.identity(3,complex)
Uleakage[1:3,1:3] = rot
leakageOp = unitary_to_gmgate(Uleakage)
#print(Uleakage)

#Guess of a model w/just unitary leakage
mdl_3level_guess = mdl_3level_ideal.copy()
mdl_3level_guess['Gi'] = np.dot(leakageOp, mdl_3level_guess['Gi'])
#mdl_3level_guess['Gx'] = np.dot(leakageOp, mdl_3level_guess['Gx'])
#mdl_3level_guess['Gy'] = np.dot(leakageOp, mdl_3level_guess['Gy'])

#Actual model used for data generation (some depolarization too)
mdl_3level_noisy = mdl_3level_ideal.depolarize(op_noise=0.005, spam_noise=0.01)
mdl_3level_noisy['Gi'] = np.dot(leakageOp, mdl_3level_noisy['Gi'])
#mdl_3level_noisy['Gx'] = np.dot(leakageOp, mdl_3level_noisy['Gx'])
#mdl_3level_noisy['Gy'] = np.dot(leakageOp, mdl_3level_noisy['Gy'])

In [6]:
#print(mdl_3level_guess)

In [7]:
# get sequences using expected model
generate_fiducials = False

if generate_fiducials:
    prepfids, measfids = pygsti.algorithms.generate_fiducials(
        mdl_3level_guess, omitIdentity=False, maxFidLength=4, verbosity=4)
    pygsti.io.write_circuit_list("example_files/leakage_prepfids.txt", prepfids)
    pygsti.io.write_circuit_list("example_files/leakage_measfids.txt", measfids)

In [8]:
prepfids = pygsti.io.load_circuit_list("example_files/leakage_prepfids.txt")
measfids = pygsti.io.load_circuit_list("example_files/leakage_measfids.txt")
germs = std1Q.germs
maxLengths = [1,]
expList = pc.create_lsgst_circuits(mdl_3level_noisy, prepfids, measfids, germs, maxLengths)
ds = pc.simulate_data(mdl_3level_noisy, expList, 1000, 'binomial', seed=1234)

In [9]:
results_2level = pygsti.run_stdpractice_gst(ds, mdl_2level_ideal, prepfids, measfids,
                                           germs, maxLengths, modes="CPTP", verbosity=3)
results_3level = pygsti.run_stdpractice_gst(ds, mdl_3level_ideal, prepfids, measfids,
                                           germs, maxLengths, modes="CPTP,True",
                                           modelsToTest={'True': mdl_3level_noisy}, 
                                           verbosity=4, advancedOptions={'all': {'tolerance': 1e-2}})

--- Circuit Creation ---
   280 sequences created
-- Std Practice:  Iter 1 of 1  (CPTP) --: 
  --- Iterative MLGST: Iter 1 of 1  280 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 801.665 (280 data params - 31 model params = expected mean of 249; p-value = 0)
    Completed in 8.6s
    2*Delta(log(L)) = 809.696
    Iteration 1 took 8.7s
    
    Switching to ML objective (last iteration)
    --- MLGST ---
      Maximum log(L) = 398.424 below upper bound of -421916
        2*Delta(log(L)) = 796.848 (280 data params - 31 model params = expected mean of 249; p-value = 0)
      Completed in 1.3s
    2*Delta(log(L)) = 796.848
    Final MLGST took 1.3s
    
  Iterative MLGST Total Time: 10.0s
--- Circuit Creation ---
   280 sequences created
-- Std Practice:  Iter 1 of 2  (CPTP) --: 
  --- Iterative MLGST: Iter 1 of 1  280 operation sequences ---: 
    --- Minimum Chi^2 GST ---
      bulk_evaltree: created initial tree (280 strs) in 0s
      bulk_evaltree: split tr

-- Std Practice:  Iter 2 of 2  (True) --: 
      -- Performing 'stdgaugeopt' gauge optimization on True estimate --


In [10]:
pygsti.report.construct_standard_report(
    {'two-level': results_2level, 'three-level': results_3level},
    "Leakage Example Report"
).write_html('example_files/leakage_report')

Running idle tomography
Computing switchable properties


/Users/enielse/pyGSTi/pygsti/report/factory.py:1236: UserWarning:

Idle tomography failed:
operands could not be broadcast together with shapes (9,) (16,) 



Found standard clifford compilation from std1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1303: UserWarning:

PDF output is not available for reports with multiple datasets



Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.


In [11]:
#try a different basis:
gm_basis = pygsti.obj.Basis.cast('gm',9)
   
leakage_basis_mxs = [ np.sqrt(2)/3*(np.sqrt(3)*gm_basis[0] + 0.5*np.sqrt(6)*gm_basis[8]),
                      gm_basis[1], gm_basis[4], gm_basis[7],
                     gm_basis[2], gm_basis[3], gm_basis[5], gm_basis[6],
                     1/3*(np.sqrt(3)*gm_basis[0] - np.sqrt(6)*gm_basis[8]) ]
#for mx in leakage_basis_mxs:
#    pygsti.tools.print_mx(mx)

check = np.zeros( (9,9), complex)
for i,m1 in enumerate(leakage_basis_mxs):
    for j,m2 in enumerate(leakage_basis_mxs):
        check[i,j] = np.trace(np.dot(m1,m2))
assert(np.allclose(check, np.identity(9,complex)))

leakage_basis = pygsti.obj.ExplicitBasis(leakage_basis_mxs, name="LeakageBasis",  
                                        longname="2+1 level leakage basis", real=True,
                                        labels=['I','X','Y','Z','LX0','LX1','LY0','LY1','L'])

def changebasis_3level_model(mdl):
    new_mdl = mdl.copy()
    new_mdl.preps['rho0'] = pygsti.obj.FullSPAMVec(
        pygsti.tools.change_basis(mdl.preps['rho0'].todense(), gm_basis, leakage_basis))
    new_mdl.povms['Mdefault'] = pygsti.obj.UnconstrainedPOVM(
        [('0', pygsti.tools.change_basis(mdl.povms['Mdefault']['0'].todense(), gm_basis, leakage_basis)),
         ('1', pygsti.tools.change_basis(mdl.povms['Mdefault']['1'].todense(), gm_basis, leakage_basis))])
    
    for lbl,op in mdl.operations.items():
        new_mdl.operations[lbl] = pygsti.obj.FullDenseOp(
            pygsti.tools.change_basis(op.todense(), gm_basis, leakage_basis))
    new_mdl.basis = leakage_basis
    return new_mdl

def changebasis_3level_results(results):
    new_results = results.copy()
    for estlbl,est in results.estimates.items():
        for mlbl,mdl in est.models.items():
            if isinstance(mdl,(list,tuple)): #assume a list/tuple of models
                new_results.estimates[estlbl].models[mlbl] = \
                    [ changebasis_3level_model(m) for m in mdl ]
            else:
                new_results.estimates[estlbl].models[mlbl] = changebasis_3level_model(mdl)
    return new_results
    

In [12]:
results_3level_leakage_basis = changebasis_3level_results( results_3level )    

In [13]:
pygsti.report.construct_standard_report(
    {'two-level': results_2level, 'three-level': results_3level_leakage_basis},
    "Leakage Example Report"
).write_html('example_files/leakage_report')

Running idle tomography
Computing switchable properties
Found standard clifford compilation from std1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1236: UserWarning:

Idle tomography failed:
operands could not be broadcast together with shapes (9,) (16,) 

/Users/enielse/pyGSTi/pygsti/report/factory.py:1303: UserWarning:

PDF output is not available for reports with multiple datasets



Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.


Open the report [here](example_files/leakage_report/main.html)

In [14]:
# use "kite" density-matrix structure
def to_2plus1_superop(superop_2level):
    ret = np.zeros((5,5),'d')
    ret[0:4,0:4] = superop_2level
    ret[4,4] = 1.0 #leave leakage population where it is
    return ret

#Tack on a single extra "0" for the 5-th dimension corresponding
# to the classical leakage level population.
rho0 = np.concatenate( (mdl_2level_ideal.preps['rho0'],[[0]]), axis=0)
E0 = np.concatenate( (mdl_2level_ideal.povms['Mdefault']['0'],[[0]]), axis=0)
E1 = np.concatenate( (mdl_2level_ideal.povms['Mdefault']['1'],[[0]]), axis=0)


sslbls = pygsti.obj.StateSpaceLabels([('Qubit',),('Leakage',)],[(4,),(1,)])
mdl_2plus1_ideal = pygsti.obj.ExplicitOpModel(sslbls, 'gm')
mdl_2plus1_ideal['rho0'] = rho0
mdl_2plus1_ideal['Mdefault'] = pygsti.obj.UnconstrainedPOVM([('0',E0),('1',E1)])

mdl_2plus1_ideal['Gi'] = to_2plus1_superop(mdl_2level_ideal['Gi'])
mdl_2plus1_ideal['Gx'] = to_2plus1_superop(mdl_2level_ideal['Gi'])
mdl_2plus1_ideal['Gy'] = to_2plus1_superop(mdl_2level_ideal['Gi'])

In [15]:
results_2plus1 = pygsti.run_long_sequence_gst(ds, mdl_2plus1_ideal, prepfids, measfids,
                                             germs, maxLengths, verbosity=3,
                                             advancedOptions={"starting point": "target"})

--- Circuit Creation ---
   280 sequences created
--- Iterative MLGST: Iter 1 of 1  280 operation sequences ---: 
  --- Minimum Chi^2 GST ---
    bulk_evaltree: created initial tree (280 strs) in 0s
    bulk_evaltree: split tree (1 subtrees) in 0s
    Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
     groups of ~1 procs each, to distribute over 90 params (taken as 1 param groups of ~90 params).
    --- Outer Iter 0: norm_f = 1.10463e+09, mu=1, |x|=4.24264, |J|=246913
    --- Outer Iter 1: norm_f = 106758, mu=2.30523e+06, |x|=4.23094, |J|=4045.54
    --- Outer Iter 2: norm_f = 76699.5, mu=768410, |x|=4.21158, |J|=3590.63
    --- Outer Iter 3: norm_f = 69764.7, mu=256137, |x|=4.2006, |J|=3504.09
    --- Outer Iter 4: norm_f = 68704, mu=85378.9, |x|=4.19376, |J|=3476.38
    --- Outer Iter 5: norm_f = 67412.7, mu=28459.6, |x|=4.17968, |J|=3422.34
    --- Outer Iter 6: norm_f = 66403.8, mu=11408.7, |x|=4.166, |J|=3376.7
    --- Outer Iter 7: norm_

    --- Outer Iter 100: norm_f = 624.028, mu=6.93394, |x|=4.32467, |J|=10246.6
    --- Outer Iter 101: norm_f = 623.818, mu=6.8888, |x|=4.32114, |J|=10218.6
    --- Outer Iter 102: norm_f = 623.629, mu=6.88769, |x|=4.32045, |J|=10195.9
    --- Outer Iter 103: norm_f = 623.459, mu=6.9114, |x|=4.32287, |J|=10173.4
    --- Outer Iter 104: norm_f = 623.308, mu=7.31403, |x|=4.32875, |J|=10148.8
    --- Outer Iter 105: norm_f = 623.125, mu=7.90122, |x|=4.3379, |J|=10119.4
    --- Outer Iter 106: norm_f = 622.938, mu=9.174, |x|=4.35033, |J|=10086.6
    --- Outer Iter 107: norm_f = 622.607, mu=9.64888, |x|=4.36504, |J|=10058.8
    --- Outer Iter 108: norm_f = 622.501, mu=15.1712, |x|=4.3845, |J|=10029.4
    --- Outer Iter 109: norm_f = 621.23, mu=13.3092, |x|=4.39997, |J|=10069.7
    --- Outer Iter 110: norm_f = 620.384, mu=9.13225, |x|=4.4113, |J|=10119.6
    --- Outer Iter 111: norm_f = 620.268, mu=23.4506, |x|=4.43217, |J|=10096.4
    --- Outer Iter 112: norm_f = 619.264, mu=14.5716, |x|=4.

    --- Outer Iter 205: norm_f = 603.692, mu=0.570793, |x|=8.12603, |J|=35502.8
    --- Outer Iter 206: norm_f = 603.687, mu=0.553197, |x|=8.17775, |J|=36122.7
    --- Outer Iter 207: norm_f = 603.683, mu=0.536052, |x|=8.22946, |J|=36749.3
    --- Outer Iter 208: norm_f = 603.678, mu=0.519349, |x|=8.28117, |J|=37382.4
    --- Outer Iter 209: norm_f = 603.674, mu=0.503077, |x|=8.33284, |J|=38021.7
    --- Outer Iter 210: norm_f = 603.67, mu=0.487227, |x|=8.38448, |J|=38667.2
    --- Outer Iter 211: norm_f = 603.666, mu=0.471789, |x|=8.43607, |J|=39318.6
    --- Outer Iter 212: norm_f = 603.662, mu=0.456753, |x|=8.48759, |J|=39975.8
    --- Outer Iter 213: norm_f = 603.658, mu=0.44211, |x|=8.53903, |J|=40638.4
    --- Outer Iter 214: norm_f = 603.655, mu=0.427849, |x|=8.59039, |J|=41306.4
    --- Outer Iter 215: norm_f = 603.651, mu=0.413962, |x|=8.64164, |J|=41979.5
    --- Outer Iter 216: norm_f = 603.648, mu=0.400438, |x|=8.69277, |J|=42657.5
    --- Outer Iter 217: norm_f = 603.645, 

    --- Outer Iter 53: norm_f = 301.178, mu=0.0658253, |x|=8.65523, |J|=30983.7
    Least squares message = Both actual and predicted relative reductions in the sum of squares are at most 1e-06
    Maximum log(L) = 301.178 below upper bound of -421916
      2*Delta(log(L)) = 602.356 (280 data params - 65 model params = expected mean of 215; p-value = 0)
    Completed in 5.1s
  2*Delta(log(L)) = 602.356
  Final MLGST took 5.1s
  
Iterative MLGST Total Time: 28.7s
    -- Performing 'go0' gauge optimization on default estimate --


/Users/enielse/pyGSTi/pygsti/algorithms/gaugeopt.py:242: UserWarning:

No gauge group specified, so no gauge optimization performed.



In [16]:
pygsti.report.construct_standard_report(
    {'two-level': results_2level, 'three-level': results_3level_leakage_basis,
     'two+one level': results_2plus1},
    "Leakage Example Report"
).write_html('example_files/leakage_report', autosize='none')

Running idle tomography
Computing switchable properties


/Users/enielse/pyGSTi/pygsti/report/factory.py:1236: UserWarning:

Idle tomography failed:
operands could not be broadcast together with shapes (9,) (16,) 



Found standard clifford compilation from std1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1303: UserWarning:

PDF output is not available for reports with multiple datasets

/Users/enielse/pyGSTi/pygsti/tools/matrixtools.py:613: UserWarning:

Cannot construct a real log: unpaired negative real eigenvalues: [(-1.0084172062338246+0j)]

/Users/enielse/pyGSTi/pygsti/report/reportables.py:1090: UserWarning:

Truncating imaginary logarithm!

/Users/enielse/pyGSTi/pygsti/tools/optools.py:1239: UserWarning:

Near-identity matrix log failed; falling back to approximate log for logGTi error generator:
Failed to construct a real logarithm! This is probably because M is not near the identity.
Its eigenvalues are: [-1.00841721+0.j          0.01252335+0.97927003j  0.01252335-0.97927003j
  1.00037875+0.j          1.00405493+0.j        ]

/Users/enielse/pyGSTi/pygsti/tools/optools.py:1243: UserWarning:

Falling back to approximate log for logGTi error generator



Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.


Open the report [here](example_files/leakage_report/main.html)